In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import time
import copy

# ==========================================
# 0. SETUP
# ==========================================
# Ensure your first script is saved as 'data_preprocessing.py'
from data_preprocessing import get_dataloaders

# ==========================================
# 1. CONFIGURATION
# ==========================================
BATCH_SIZE = 32
# NUM_CLASSES is now dynamic based on the dataset loader
EPOCHS = 25
LEARNING_RATE = 0.0001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_SAVE_PATH = "resnet50_best_model_new_final_nodel.pth"

# ==========================================
# 2. MODEL ARCHITECTURE
# ==========================================
def get_resnet_model(num_classes):
    print(f"🏗️ Initializing ResNet-50 for {num_classes} classes...")
    # Load Pre-trained Weights (ImageNet)
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    
    # Freeze initial layers to retain basic features
    for param in model.parameters():
        param.requires_grad = False
        
    # Unfreeze deep layers (Layer 3 and 4) for fine-grained adaptation
    for param in model.layer3.parameters():
        param.requires_grad = True
    for param in model.layer4.parameters():
        param.requires_grad = True
        
    # Custom Classification Head
    in_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(in_features, 1024),
        nn.ReLU(),
        nn.Dropout(0.5), # Strong dropout
        nn.Linear(1024, num_classes)
    )
    return model

# ==========================================
# 3. TRAINING FUNCTION
# ==========================================
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=15):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    print(f"🚀 Training on device: {DEVICE}")

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dataloader = train_loader
            else:
                model.eval()   # Set model to evaluate mode
                dataloader = val_loader

            running_loss = 0.0
            running_corrects = 0
            total_samples = 0

            # Iterate over data
            for inputs, labels in dataloader:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                total_samples += inputs.size(0)

            epoch_loss = running_loss / total_samples
            epoch_acc = running_corrects.double() / total_samples

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if it's the best one so far
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), MODEL_SAVE_PATH)
                print(f"🌟 New Best Model Saved! (Acc: {best_acc:.4f})")
            
            # Step the scheduler at the end of validation
            if phase == 'val':
                scheduler.step(epoch_loss)

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best Validation Accuracy: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# ==========================================
# 4. EVALUATION FUNCTION
# ==========================================
def evaluate_model(model, test_loader):
    print("\n🔍 Starting Final Evaluation on Test Set...")
    model.eval()
    running_corrects = 0
    total_samples = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            running_corrects += torch.sum(preds == labels.data)
            total_samples += inputs.size(0)
            
    acc = running_corrects.double() / total_samples
    print(f"🏆 Final Test Accuracy: {acc:.4f}")

# ==========================================
# 5. MAIN EXECUTION
# ==========================================
if __name__ == "__main__":
    try:
        # 1. Load Data
        # IMPORTANT: Unpacking 5 values now (added classes and mapping)
        train_dl, val_dl, test_dl, selected_classes, label_map = get_dataloaders(BATCH_SIZE)
        
        # Calculate dynamic num_classes
        dynamic_num_classes = len(selected_classes)
        print(f"📊 Dataset ready. Training on {dynamic_num_classes} classes.")
        
        # 2. Initialize Model with correct number of classes
        resnet_model = get_resnet_model(dynamic_num_classes)
        resnet_model = resnet_model.to(DEVICE)
        
        # 3. Setup Training Configuration
        criterion = nn.CrossEntropyLoss()
        
        optimizer = optim.Adam(
            resnet_model.parameters(), 
            lr=LEARNING_RATE, 
            weight_decay=1e-4
        )
        
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.1, patience=3, verbose=True
        )
        
        # 4. Start Training
        best_model = train_model(
            resnet_model, 
            train_dl, 
            val_dl, 
            criterion, 
            optimizer, 
            scheduler, 
            num_epochs=EPOCHS
        )
        
        # 5. Final Test Evaluation
        evaluate_model(best_model, test_dl)
        
        print("✅ Process Completed Successfully.")
        
    except KeyboardInterrupt:
        print("\n🛑 Training interrupted by user.")
    except Exception as e:
        print(f"\n❌ An error occurred: {e}")
        # Print stack trace for debugging if needed
        import traceback
        traceback.print_exc()

🚀 Loading 'tanganke/stanford_cars' from Hugging Face Hub...
📊 Total classes in dataset: 196
🎯 Selected 20 random classes: [6, 7, 8, 22, 23, 26, 28, 35, 55, 57, 59, 62, 70, 108, 139, 151, 163, 173, 188, 189]


Filter: 100%|██████████| 8144/8144 [00:59<00:00, 136.03 examples/s]


✅ Filtered dataset size: 829 samples
📊 Total classes in dataset: 196
🎯 Selected 20 random classes: [6, 7, 8, 22, 23, 26, 28, 35, 55, 57, 59, 62, 70, 108, 139, 151, 163, 173, 188, 189]


Filter: 100%|██████████| 8041/8041 [01:00<00:00, 131.84 examples/s]


✅ Filtered dataset size: 820 samples


Filter: 100%|██████████| 8041/8041 [00:30<00:00, 262.53 examples/s]


✅ Data Split: 663 Train | 166 Val | 820 Test
📌 Classes remapped to range: 0-19
📊 Dataset ready. Training on 20 classes.
🏗️ Initializing ResNet-50 for 20 classes...


C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


🚀 Training on device: cuda
Epoch 1/25
----------
Train Loss: 2.9346 Acc: 0.1146
Val Loss: 2.7002 Acc: 0.2831
🌟 New Best Model Saved! (Acc: 0.2831)

Epoch 2/25
----------
Train Loss: 2.4122 Acc: 0.4103
Val Loss: 1.9019 Acc: 0.4458
🌟 New Best Model Saved! (Acc: 0.4458)

Epoch 3/25
----------
Train Loss: 1.6603 Acc: 0.5988
Val Loss: 1.3999 Acc: 0.6807
🌟 New Best Model Saved! (Acc: 0.6807)

Epoch 4/25
----------
Train Loss: 0.9340 Acc: 0.8190
Val Loss: 0.8998 Acc: 0.7651
🌟 New Best Model Saved! (Acc: 0.7651)

Epoch 5/25
----------
Train Loss: 0.4588 Acc: 0.9216
Val Loss: 0.6963 Acc: 0.8373
🌟 New Best Model Saved! (Acc: 0.8373)

Epoch 6/25
----------
Train Loss: 0.2462 Acc: 0.9698
Val Loss: 0.6677 Acc: 0.8133

Epoch 7/25
----------
Train Loss: 0.1377 Acc: 0.9849
Val Loss: 0.7134 Acc: 0.7831

Epoch 8/25
----------
Train Loss: 0.0934 Acc: 0.9910
Val Loss: 0.5752 Acc: 0.8313

Epoch 9/25
----------
Train Loss: 0.0624 Acc: 0.9940
Val Loss: 0.6772 Acc: 0.8012

Epoch 10/25
----------
Train Loss: 0